In [1]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import xgboost as xgb
from sklearn.utils import shuffle
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import TimeSeriesSplit, cross_val_score, GridSearchCV, train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier


from itertools import combinations


In [2]:
npa = np.load('../input/taskk2/class_a.npy')
npb = np.load('../input/taskk2/class_b.npy')
print(npb.shape)
print(npa.shape)

(1000, 40, 60)
(1000, 40, 60)


Уменьшаем размерность данных с 3 до 2, чтобы в дальнейшем было удобнее переводить в формат pandas.DataFrame

In [3]:
npa2 = npa.transpose(2,0,1).reshape(1000,-1)
npb2 = npb.transpose(2,0,1).reshape(1000,-1)

In [4]:
col_names =['f'+str(i) for i in range(2400)]
df_a = pd.DataFrame(npa2, columns=col_names) 
df_a.head()

,f0,f1,f2,f3,f4,f5,f6,f7,f8,f9,...,f2390,f2391,f2392,f2393,f2394,f2395,f2396,f2397,f2398,f2399
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
df_b = pd.DataFrame(npb2, columns=col_names)
df_b.head()

,f0,f1,f2,f3,f4,f5,f6,f7,f8,f9,...,f2390,f2391,f2392,f2393,f2394,f2395,f2396,f2397,f2398,f2399
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Добавляем новый признак - target - который отвечает за принадлежность классу: 0 - класс A, 1 - класс B.

In [6]:
df_a['target'] = 0
df_b['target'] = 1

Соединяем данные обоих классов  в один data_train, случайно перемешивая.

In [9]:
data_train = df_a.append(df_b)
data_train = shuffle(data_train).reset_index(drop=True)
data_train.head()

,f0,f1,f2,f3,f4,f5,f6,f7,f8,f9,...,f2391,f2392,f2393,f2394,f2395,f2396,f2397,f2398,f2399,target
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1


Делим данные на валидационный и тренировочный датасет(70% на тренировку, 30% на валидацию)

In [10]:
y = data_train['target']
X = data_train.drop('target', axis=1)

# Split into train & validation set
X_train, X_val, y_train, y_val = train_test_split(X, y, train_size=0.7)

Далее применяем модели. Так как признаки сильно разрежены(по процентам в районе 94-95%), то применим логистическую регрессию, которая лучше справляется именно с таким типом данных. Метрикой классификации выберем площать по ROC-кривой. 

In [11]:
lr = LogisticRegression(C=1.0, random_state=17, solver='lbfgs', max_iter=500).fit(X_train, y_train)

y_pred = lr.predict_proba(X_val)[:, 1]
score = roc_auc_score(y_val, y_pred)
print(score)

0.8059440559440558


Теперь подберем наилучший гиперпараметр(параметр "C" будем из диапазона от -2 до 2) для данной модели обучения. Сделаем это при помощи GridSearch.


In [12]:
c_values = np.logspace(-2, 2, 20)

logit_grid_searcher = GridSearchCV(estimator=lr, param_grid={'C': c_values},
                                  scoring='roc_auc', n_jobs=4, cv=5, verbose=1)

In [13]:
logit_grid_searcher.fit(X_train, y_train)

Fitting 5 folds for each of 20 candidates, totalling 100 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    9.8s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:   25.4s finished


GridSearchCV(cv=5, estimator=LogisticRegression(max_iter=500, random_state=17),
             n_jobs=4,
             param_grid={'C': array([1.00000000e-02, 1.62377674e-02, 2.63665090e-02, 4.28133240e-02,
       6.95192796e-02, 1.12883789e-01, 1.83298071e-01, 2.97635144e-01,
       4.83293024e-01, 7.84759970e-01, 1.27427499e+00, 2.06913808e+00,
       3.35981829e+00, 5.45559478e+00, 8.85866790e+00, 1.43844989e+01,
       2.33572147e+01, 3.79269019e+01, 6.15848211e+01, 1.00000000e+02])},
             scoring='roc_auc', verbose=1)

In [14]:
logit_grid_searcher.best_score_, logit_grid_searcher.best_params_

(0.7779792763753813, {'C': 0.016237767391887217})

Получили, что при использовании кросс-валидации точность увеличилась. А лучшее значение показателя равно C = 0.01

In [23]:
lr = LogisticRegression(C= 0.016237767, random_state=17, solver='lbfgs', max_iter=500).fit(X_train, y_train)
y_pred = lr.predict(X_val)
score = roc_auc_score(y_val, y_pred)
print(score)
cv_scores1 = cross_val_score(lr, X_val, y_val, cv=5, scoring='roc_auc', n_jobs=4)
cv_scores1,cv_scores1.mean()


0.7330742505901742


(array([0.80478309, 0.81982735, 0.76009468, 0.78084099, 0.7695628 ]),
 0.7870217813722292)

Хоть деревья решений не так хорошо справляются с разреженными данными, как линейные модели, но все же рассмотрим такую модель как GradientBoostingClassifier. Сразу при помощи GridSearch найдем оптимальные гиперпараметры.

In [18]:
parameters = {
    "learning_rate": [0.01,0.025,0.05, 0.075, 0.1],
    "max_depth":[80,90,100],
    "max_features":['sqrt','log2'],
    "n_estimators":[200, 250, 300]
}

grid = GridSearchCV(GradientBoostingClassifier(random_state=0), parameters, cv=3, n_jobs=-1, 
                    return_train_score=True, scoring = 'roc_auc')
grid.fit(X_train,y_train)

print(grid.best_estimator_)

GradientBoostingClassifier(learning_rate=0.05, max_depth=80,
                           max_features='sqrt', n_estimators=300,
                           random_state=0)


In [19]:
clf = GradientBoostingClassifier(learning_rate=0.05, max_depth=80,max_features='sqrt',n_estimators=300, random_state=0)
clf.fit(X_train,y_train)

y_pred = clf.predict(X_val)
score = roc_auc_score(y_val, y_pred)
print(score)
cv_scores1 = cross_val_score(clf, X_val, y_val, cv=5, scoring='roc_auc', n_jobs=4)
cv_scores1,cv_scores1.mean()

0.7755667898980001


(array([0.8320356 , 0.84823169, 0.8166249 , 0.8512949 , 0.83361181]),
 0.8363597784474781)

Мы получили, что GradientBoostingClassifier лучше справляется с задачей классификации по сравнению с логистической регрессией.